In [1]:
import datetime
import tqdm

import catboost
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

from py import sqlite_utils as utils

In [2]:
conn = utils.connect_sqlite()

In [3]:
df_train = utils.execute_sql_query("""
SELECT *
FROM power_features_train
""", conn=conn)
df_test = utils.execute_sql_query("""
SELECT *
FROM power_features_test
""", conn=conn)

## Baseline - LAG(y, 1) as prediction

In [4]:
preds = pd.concat([df_train[['hour', 'y_kw_mean_lag1']], df_test[['hour', 'y_kw_mean_lag1']]])\
    .rename({'y_kw_mean_lag1': 'pred_value'}, axis=1).reset_index(drop=True).dropna()

In [5]:
utils.insert_predictions(preds, 'baseline_lag1', conn=conn)

Model version is 0
Inserted 34162 rows


In [6]:
preds = pd.concat([df_train[['hour', 'y_kw_mean']], df_test[['hour', 'y_kw_mean']]])\
    .rename({'y_kw_mean': 'pred_value'}, axis=1).reset_index(drop=True)
preds['pred_value'] = preds['pred_value'].ffill().shift(1)
preds = preds.dropna()

In [7]:
utils.insert_predictions(preds, 'baseline_ffill_lag1', conn=conn)

Model version is 0
Inserted 34406 rows


## Baseline CatBoost model

In [8]:
model = catboost.CatBoostRegressor(
    loss_function='MAE',
    iterations=5000,
    random_seed=338
)

In [9]:
X_train = df_train.iloc[:, 2:].values
y_train = df_train.iloc[:, 1].values

X_test = df_test.iloc[:, 2:].values

In [10]:
model.fit(X_train, y_train, silent=True)

In [11]:
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)
preds = pd.concat([
    pd.DataFrame({'hour': df_train['hour'], 'pred_value': train_preds}),
    pd.DataFrame({'hour': df_test['hour'], 'pred_value': test_preds})
])

In [12]:
utils.insert_predictions(preds, 'catboost_baseline', conn=conn)

Model version is 0
Inserted 34407 rows


Base params for grid search:

In [13]:
{k: v for k, v in model.get_all_params().items() if k in ['depth', 'learning_rate', 'l2_leaf_reg']}

{'l2_leaf_reg': 3, 'depth': 6, 'learning_rate': 0.029999999329447743}

## CatBoost parameter optimization

In [14]:
def calculate_cv_score(X, y, params, n_splits=5):
    """
    Cross-validate CatBoostRegressor with given params on TimeTestSplit.
    Returns mean and std of per-split MAE values on eval subsets.
    """
    splitter = TimeSeriesSplit(n_splits=n_splits)
    results = []

    for train_idx, eval_idx in tqdm.tqdm(splitter.split(X)):
        X_tr, X_ev = X[train_idx], X[eval_idx]
        y_tr, y_ev = y[train_idx], y[eval_idx]

        model = catboost.CatBoostRegressor(
            loss_function='MAE',
            iterations=5000,
            random_seed=338,
            **params
        )
        model.fit(X_tr, y_tr, silent=True)
        results.append(mean_absolute_error(y_ev, model.predict(X_ev)))

    return np.mean(results), np.std(results)

In [15]:
grid = []
for depth in [5, 6, 7, 8]:
    for lr in [0.01, 0.02, 0.03, 0.05, 0.07]:
        for l2 in [1, 3, 10, 30]:
            grid.append({'depth': depth, 'learning_rate': lr, 'l2_leaf_reg': l2})

result_list = []
for p in grid:
    mae_mean, mae_std = calculate_cv_score(X_train, y_train, p, n_splits=4)
    print(p, '|', f'MAE = {mae_mean:.4f} +- {mae_std:.4f}')
    res = {k: v for k, v in p.items()}
    res['mae'] = mae_mean
    result_list.append(res)

4it [01:40, 25.00s/it]


{'depth': 5, 'learning_rate': 0.01, 'l2_leaf_reg': 1} | MAE = 0.3628 +- 0.0405


4it [01:38, 24.75s/it]


{'depth': 5, 'learning_rate': 0.01, 'l2_leaf_reg': 3} | MAE = 0.3584 +- 0.0368


4it [01:36, 24.13s/it]


...

{'depth': 8, 'learning_rate': 0.07, 'l2_leaf_reg': 3} | MAE = 0.4076 +- 0.0653


4it [06:16, 94.19s/it]


{'depth': 8, 'learning_rate': 0.07, 'l2_leaf_reg': 10} | MAE = 0.3985 +- 0.0559


4it [06:16, 94.25s/it]

{'depth': 8, 'learning_rate': 0.07, 'l2_leaf_reg': 30} | MAE = 0.3910 +- 0.0543


In [16]:
pd.DataFrame(result_list).sort_values('mae')[:5]

,depth,learning_rate,l2_leaf_reg,mae
61,8,0.01,3,0.355795
43,7,0.01,30,0.355883
23,6,0.01,30,0.356576
60,8,0.01,1,0.357170
63,8,0.01,30,0.357557


In [17]:
best_params = {k: v for k,v in min(result_list, key=lambda x: x['mae']).items() if k != 'mae'}
best_params

{'depth': 8, 'learning_rate': 0.01, 'l2_leaf_reg': 3}

In [20]:
model = catboost.CatBoostRegressor(
    loss_function='MAE',
    iterations=5000,
    random_seed=338,
    **best_params
)

In [21]:
model.fit(X_train, y_train, silent=True)

In [22]:
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)
preds = pd.concat([
    pd.DataFrame({'hour': df_train['hour'], 'pred_value': train_preds}),
    pd.DataFrame({'hour': df_test['hour'], 'pred_value': test_preds})
])

In [23]:
utils.insert_predictions(preds, 'catboost', conn=conn)

Model version is 0
Inserted 34407 rows


In [25]:
model.save_model('data/catboost_model0.cbm')

In [26]:
conn.close()